In [1]:
import os
import pandas as pd
import spacy
import csv
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example


2023-05-30 16:54:00.536422: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 16:54:00.574112: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 16:54:01.309455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

2023-05-30 16:54:07.277344: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
# data
from subprocess import check_output
print(check_output(["ls", "/home/jovyan/work/data/label_data"]).decode("utf8"))

digitoday.2014.csv
digitoday.2014.dev.csv
digitoday.2014.train.csv
digitoday.2015.test.csv
ner.csv
wikipedia.test.csv



In [4]:
dframe = pd.read_csv("/home/jovyan/work/data/label_data/ner.csv")
dframe.head(30)

,Unnamed: 0,sentence_idx,shape,word,tag
0,0,1.0,capitalized,Thousands,O
1,1,1.0,lowercase,of,O
2,2,1.0,lowercase,demonstrators,O
3,3,1.0,lowercase,have,O
4,4,1.0,lowercase,marched,O
5,5,1.0,lowercase,through,O
6,6,1.0,capitalized,London,B-geo
7,7,1.0,lowercase,to,O
8,8,1.0,lowercase,protest,O
9,9,1.0,lowercase,the,O


In [5]:
grouped = dframe.groupby("sentence_idx").apply(lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),s["tag"].values.tolist())])
sentences = [s for s in grouped]

In [6]:
test = [[('TokyoiTower', 'O'),
  ('of', 'O'),
  ('demonstrators', 'O'),
  ('have', 'O'),
  ('marched', 'O'),
  ('through', 'O'),
  ('London', 'B-geo'),
  ('to', 'O'),
  ('protest', 'O')],[
  ('the', 'O'),
  ('war', 'O'),
  ('in', 'O'),
  ('Iraq', 'B-geo')]]
data = []
for s in sentences:
    full_s = ""
    start_index = 0
    last_index = 0
    index_counter = 0
    entities = []
    for w in s:
        full_s += str(w[0]) + " "
        entities.append((index_counter, index_counter+len(str(w[0])),w[1]))
        index_counter += len(str(w[0])) + 1



    data.append((full_s,{"entities":entities}))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [10]:
# Initialize a new blank spaCy model
nlp = spacy.load("en_core_web_sm")

# Create the NER component
# ner = nlp.add_pipe("ner", name="ner", last=True)
ner = nlp.get_pipe("ner")

In [11]:
# Add custom tags to the NER pipeline
for _, annotations in data:
    for entity in annotations.get("entities"):
        ner.add_label(entity[2])

# Disable unnecessary pipeline components
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


In [12]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
    # Training iterations...
    for iteration in range(30):
        # Shuffle training examples before every iteration
        random.shuffle(train_data)
        losses = {}
        examples = []

        # Create Example objects from training data
        for text, annotation in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            examples.append(example)

        # Batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses)

        print("Losses:", losses)


/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "In a telephone interview to discuss the issues at ..." with entities "[(0, 2, 'O'), (3, 4, 'O'), (5, 14, 'O'), (15, 24, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 72948.6766353976}
Losses: {'ner': 53892.802191747985}
Losses: {'ner': 49373.22177781198}
Losses: {'ner': 46675.80639278384}
Losses: {'ner': 44526.610624272515}
Losses: {'ner': 43102.21695223445}
Losses: {'ner': 41834.96102580074}
Losses: {'ner': 40820.977349422516}
Losses: {'ner': 39931.7687976192}
Losses: {'ner': 38952.1496582074}
Losses: {'ner': 38330.972942701424}
Losses: {'ner': 37781.743091212}
Losses: {'ner': 37193.49617848435}
Losses: {'ner': 36633.29867627069}
Losses: {'ner': 36165.83993138419}
Losses: {'ner': 35813.38361999737}
Losses: {'ner': 35420.357336481145}
Losses: {'ner': 34975.28958016925}
Losses: {'ner': 34526.96085174388}
Losses: {'ner': 34331.751485689696}
Losses: {'ner': 34178.11955868965}
Losses: {'ner': 34029.63697604398}
Losses: {'ner': 33715.20311482853}
Losses: {'ner': 33430.42274669204}
Losses: {'ner': 33344.676770946506}
Losses: {'ner': 32884.82132727045}
Losses: {'ner': 32888.91318387674}
Losses: {'ner': 32628.017491638842}
Losses: {'ner': 3

In [14]:
# Save the trained model
output_dir = "/home/jovyan/work/data/output/spacyModel"
nlp.to_disk(output_dir)

In [18]:
# # Testing the model This is for the other parameter
# test_examples = []
# for text, annotation in test_data:
#     doc = nlp.make_doc(text)
#     example = Example.from_dict(doc, annotation)
#     test_examples.append(example)
#
# # Evaluate the model on the test set
# scores = nlp.evaluate(test_examples)
# print("Test Scores:", scores)

# This is for accuracy
test_examples = []
for text, annotation in test_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotation)
    test_examples.append(example)

# Evaluate the model on the test set
scores = nlp.evaluate(test_examples)

# Calculate accuracy for each tag
tag_accuracies = {}
for tag, values in scores['ents_per_type'].items():
    tag_accuracy = round(values['f'] * 100, 2)  # Convert F1 score to percentage and round to 2 decimal places
    tag_accuracies[tag] = tag_accuracy

# Print accuracy for each tag
print("Accuracy per Tag:")
for tag, accuracy in tag_accuracies.items():
    print("- {}: {}%".format(tag, accuracy))


Accuracy per Tag:
- B-geo: 87.68%
- O: 99.19%
- B-gpe: 94.4%
- I-geo: 79.38%
- B-tim: 90.28%
- I-tim: 79.17%
- B-org: 75.92%
- I-org: 78.81%
- B-per: 83.68%
- I-per: 87.2%
- B-art: 13.19%
- I-art: 0.0%
- I-gpe: 73.56%
- B-eve: 43.4%
- I-eve: 43.01%
- I-nat: 68.57%
- B-nat: 40.48%


In [17]:
token_accuracy = scores['token_acc']
accuracy = round(token_accuracy * 100, 2)  # Convert to percentage and round to 2 decimal places

print("Accuracy: {}%".format(accuracy))

Accuracy: 100.0%


In [16]:
entity_metrics = scores['ents_per_type']

precision_sum = 0.0
recall_sum = 0.0
f_score_sum = 0.0
num_entity_types = len(entity_metrics)

for entity_type, metrics in entity_metrics.items():
    precision_sum += metrics['p']
    recall_sum += metrics['r']
    f_score_sum += metrics['f']

general_precision = precision_sum / num_entity_types
general_recall = recall_sum / num_entity_types
general_f_score = f_score_sum / num_entity_types

print("General Precision:", general_precision)
print("General Recall:", general_recall)
print("General F-Score:", general_f_score)

General Precision: 0.7191595723387469
General Recall: 0.646973869330037
General F-Score: 0.669365490491828
